<a href="https://colab.research.google.com/github/iyioladaniel/Machine_Learning/blob/master/F_E_LogReg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

file_url = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter03/bank-full.csv'
bankData=pd.read_csv(file_url,sep=';')

from sklearn import preprocessing
print(bankData['balance'])

x=bankData[['balance']].values.astype(float)
print(bankData['balance'])

minmaxScaler=preprocessing.MinMaxScaler()
bankData['balanceTran']=minmaxScaler.fit_transform(x)
bankData.head

#adding a small numerical constant to eliminate 0 values
bankData['balanceTran']=bankData['balanceTran']+0.00001

#Let us transform values for loan data
bankData['loanTran']=1

#Giving a wight of 5 if there is no loan
bankData.loc[bankData['loan'] == 'no', 'loanTran'] = 5 
bankData.head()

#Transfor values for Housing data
bankData['houseTran']=5
bankData.loc[bankData['housing']=='no', 'houseTran']=1
print(bankData.head())

#Creating new variable
bankData['assetIndex']=bankData['balanceTran']*bankData['loanTran']*bankData['houseTran']
bankData.head()

#finding the quantile
Q1,Q2,Q3=np.quantile(bankData['assetIndex'],[.25,.5,.75])

#Create quantile from assetIndex data
bankData['assetClass'] ='Q1'
bankData.loc[(bankData['assetIndex']>=Q1) & (bankData['assetIndex']<Q2), 'assetClass']='Q2'
bankData.loc[(bankData['assetIndex']>=Q2) & (bankData['assetIndex']<Q3), 'assetClass']='Q3'
bankData.loc[(bankData['assetIndex']>=Q3), 'assetClass']='Q4'
print(bankData.head())

#Calculating total of each asset class
assetTot=bankData.groupby('assetClass')['y'].agg(assetTot='count').reset_index()
#Calculating the category wise counts
assetProp=bankData.groupby(['assetClass','y'])['y'].agg(assetCat='count').reset_index()

#Merging both the data frames
assetComb=pd.merge(assetProp, assetTot, on=['assetClass'])
assetComb['catProp']=(assetComb.assetCat/assetComb.assetTot)*100
assetComb

0        2143
1          29
2           2
3        1506
4           1
         ... 
45206     825
45207    1729
45208    5715
45209     668
45210    2971
Name: balance, Length: 45211, dtype: int64
0        2143
1          29
2           2
3        1506
4           1
         ... 
45206     825
45207    1729
45208    5715
45209     668
45210    2971
Name: balance, Length: 45211, dtype: int64
   age           job  marital  education  ...   y  balanceTran loanTran houseTran
0   58    management  married   tertiary  ...  no     0.092269        5         5
1   44    technician   single  secondary  ...  no     0.073077        5         5
2   33  entrepreneur  married  secondary  ...  no     0.072832        1         5
3   47   blue-collar  married    unknown  ...  no     0.086486        5         5
4   33       unknown   single    unknown  ...  no     0.072822        5         1

[5 rows x 20 columns]
   age           job  marital  ... houseTran assetIndex  assetClass
0   58    management  m

,assetClass,y,assetCat,assetTot,catProp
0,Q1,no,10154,11293,89.914106
1,Q1,yes,1139,11293,10.085894
2,Q2,no,9195,11311,81.292547
3,Q2,yes,2116,11311,18.707453
4,Q3,no,10316,11300,91.292035
5,Q3,yes,984,11300,8.707965
6,Q4,no,10257,11307,90.713717
7,Q4,yes,1050,11307,9.286283


In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

bankCat=pd.get_dummies(bankData[['job','marital','education','default',\
                                 'housing','loan','contact','month','poutcome']])
bankNum=bankData[['age','balance','day','duration','campaign','pdays','previous','assetIndex']]

#Normalizing data
from sklearn.preprocessing import MinMaxScaler
minmaxScaler=preprocessing.MinMaxScaler()

#Creating transformation variables
ageT1 = bankNum[['age']].values.astype(float)
dayT1 = bankNum[['day']].values.astype(float)
durT1 = bankNum[['duration']].values.astype(float)
#Transform the balance data
bankNum['ageTran'] = minmaxScaler.fit_transform(ageT1)
bankNum['dayTran'] = minmaxScaler.fit_transform(dayT1)
bankNum['durTran'] = minmaxScaler.fit_transform(durT1)

bankNum2=bankNum[['ageTran','dayTran','campaign','pdays','previous','assetIndex']]

#Concatenate num and cat values
X=pd.concat([bankCat,bankNum2], axis=1)
Y=bankData['y']
print(Y)

#split data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=100)
bankModel=LogisticRegression().fit(X_train, y_train)

prediction=bankModel.predict(X_test)
print('Accuracy of Logistic Regression model prediction on test set: {:.2f}'.format(bankModel.score(X_test,y_test)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

0         no
1         no
2         no
3         no
4         no
        ... 
45206    yes
45207    yes
45208    yes
45209     no
45210     no
Name: y, Length: 45211, dtype: object
Accuracy of Logistic Regression model prediction on test set: 0.89


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [0]:
from sklearn.metrics import confusion_matrix
confusionMatrix=confusion_matrix(y_test,prediction)
print(confusionMatrix)

from sklearn.metrics import classification_report
print(classification_report(y_test, prediction))

[[11828   157]
 [ 1306   273]]
              precision    recall  f1-score   support

          no       0.90      0.99      0.94     11985
         yes       0.63      0.17      0.27      1579

    accuracy                           0.89     13564
   macro avg       0.77      0.58      0.61     13564
weighted avg       0.87      0.89      0.86     13564

